In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import sklearn as sks
import numpy as np

In [2]:
data_file = "Resources/ASvsNonAS.csv"

df = pd.read_csv(data_file, encoding='ISO-8859-1')
df.head()

,Rk,Player,Year,Age,Tm,Lg,G,PA,AB,R,...,SF,GDP,SB,CS,BA,OBP,SLG,OPS,Pos,All Star and/or HOF (played prior to 1933 when AS Game was created)
0,1,Barry Bonds,2002,37,SFG,NL,143,612,403,117,...,2.0,4.0,9.0,2.0,0.370,0.582,0.799,1.381,*7/DH,1
1,2,Barry Bonds,2004,39,SFG,NL,147,617,373,129,...,3.0,5.0,6.0,1.0,0.362,0.609,0.812,1.422,*7/HD,1
2,3,Barry Bonds,2001,36,SFG,NL,153,664,476,129,...,2.0,5.0,13.0,3.0,0.328,0.515,0.863,1.379,*7/DH,1
3,4,Babe Ruth,1920,25,NYY,AL,142,617,458,158,...,NaN,NaN,14.0,14.0,0.376,0.532,0.847,1.379,978/3H1,1
4,5,Babe Ruth,1923,28,NYY,AL,152,699,522,151,...,NaN,NaN,17.0,21.0,0.393,0.545,0.764,1.309,97/83,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 30 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   Rk                                                                   3000 non-null   int64  
 1   Player                                                               3000 non-null   object 
 2   Year                                                                 3000 non-null   int64  
 3   Age                                                                  3000 non-null   int64  
 4   Tm                                                                   3000 non-null   object 
 5   Lg                                                                   3000 non-null   object 
 6   G                                                                    3000 non-null   int64  
 7   PA    

In [4]:
#Fill all the emapty values with 0
df = df.fillna(value=0)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 30 columns):
 #   Column                                                               Non-Null Count  Dtype  
---  ------                                                               --------------  -----  
 0   Rk                                                                   3000 non-null   int64  
 1   Player                                                               3000 non-null   object 
 2   Year                                                                 3000 non-null   int64  
 3   Age                                                                  3000 non-null   int64  
 4   Tm                                                                   3000 non-null   object 
 5   Lg                                                                   3000 non-null   object 
 6   G                                                                    3000 non-null   int64  
 7   PA    

In [6]:
#Check to see if there any NaN values
df.isna()

,Rk,Player,Year,Age,Tm,Lg,G,PA,AB,R,...,SF,GDP,SB,CS,BA,OBP,SLG,OPS,Pos,All Star and/or HOF (played prior to 1933 when AS Game was created)
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2996,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2997,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2998,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [7]:
#drop the name player column
df.drop(columns = ["Rk", "Year", "Player", "Age","Tm", "Lg", "Pos"], axis = 1, inplace=True)
df.head()

,G,PA,AB,R,H,2B,3B,HR,RBI,BB,...,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,All Star and/or HOF (played prior to 1933 when AS Game was created)
0,143,612,403,117,149,31,2,46,110,198,...,0.0,2.0,4.0,9.0,2.0,0.370,0.582,0.799,1.381,1
1,147,617,373,129,135,27,3,45,101,232,...,0.0,3.0,5.0,6.0,1.0,0.362,0.609,0.812,1.422,1
2,153,664,476,129,156,32,2,73,137,177,...,0.0,2.0,5.0,13.0,3.0,0.328,0.515,0.863,1.379,1
3,142,617,458,158,172,36,9,54,135,150,...,5.0,0.0,0.0,14.0,14.0,0.376,0.532,0.847,1.379,1
4,152,699,522,151,205,45,13,41,130,170,...,3.0,0.0,0.0,17.0,21.0,0.393,0.545,0.764,1.309,1


In [8]:
#Rename the All Star and/or HOF (played prior to 1933 when AS Game was created) column to Classification
df.rename(columns={"All Star and/or HOF (played prior to 1933 when AS Game was created)":"Classification"}, inplace=True)

In [9]:
df.columns

Index(['G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB', 'IBB', 'SO',
       'HBP', 'SH', 'SF', 'GDP', 'SB', 'CS', 'BA', 'OBP', 'SLG', 'OPS',
       'Classification'],
      dtype='object')

In [10]:
#display dataframe
df.head()

,G,PA,AB,R,H,2B,3B,HR,RBI,BB,...,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,Classification
0,143,612,403,117,149,31,2,46,110,198,...,0.0,2.0,4.0,9.0,2.0,0.370,0.582,0.799,1.381,1
1,147,617,373,129,135,27,3,45,101,232,...,0.0,3.0,5.0,6.0,1.0,0.362,0.609,0.812,1.422,1
2,153,664,476,129,156,32,2,73,137,177,...,0.0,2.0,5.0,13.0,3.0,0.328,0.515,0.863,1.379,1
3,142,617,458,158,172,36,9,54,135,150,...,5.0,0.0,0.0,14.0,14.0,0.376,0.532,0.847,1.379,1
4,152,699,522,151,205,45,13,41,130,170,...,3.0,0.0,0.0,17.0,21.0,0.393,0.545,0.764,1.309,1


In [11]:
#drop team and league columns
#df = df.drop(columns = ['Tm', 'Lg', ])
#df.head()

In [12]:
#drop position column
#df = df.drop(columns=['Pos'])
#df.head()

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X = df.drop(columns = ['Classification'])
y = df['Classification']

In [15]:
#split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 2)

In [16]:
#create logistic regression model and print model score
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train, y_train)
classifier.score(X_test, y_test)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8666666666666667

In [17]:
# scale the data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [18]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
# train Logistic Regression model on scaled data and print model score
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(X_train_scaled, y_train)
classifier.score(X_test_scaled, y_test)

0.868

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   G               3000 non-null   int64  
 1   PA              3000 non-null   int64  
 2   AB              3000 non-null   int64  
 3   R               3000 non-null   int64  
 4   H               3000 non-null   int64  
 5   2B              3000 non-null   int64  
 6   3B              3000 non-null   int64  
 7   HR              3000 non-null   int64  
 8   RBI             3000 non-null   int64  
 9   BB              3000 non-null   int64  
 10  IBB             3000 non-null   float64
 11  SO              3000 non-null   int64  
 12  HBP             3000 non-null   float64
 13  SH              3000 non-null   float64
 14  SF              3000 non-null   float64
 15  GDP             3000 non-null   float64
 16  SB              3000 non-null   float64
 17  CS              3000 non-null   f

In [21]:
loops = list(df.columns.values)
#print(loops)

loops.remove('Classification')
#print(loops)

stats = []

print("Enter integers only.")
for i in loops:
    column = i
    
    user_input = (input(f"What is the stat for {column}? "))
    stats.append(user_input)
    break
    
print("Thank you.")

Enter integers only.
What is the stat for G? 3.1
What is the stat for PA? 3.2
What is the stat for AB? 4.1
What is the stat for R? 1.2
What is the stat for H? 3.2
What is the stat for 2B? 4.3
What is the stat for 3B? 2.1
What is the stat for HR? 5
What is the stat for RBI? 2.5
What is the stat for BB? 3.
What is the stat for IBB? 1.2
What is the stat for SO? 3.1
What is the stat for HBP? 4.1
What is the stat for SH? 7.1
What is the stat for SF? 1.2
What is the stat for GDP? 1.2
What is the stat for SB? 3.1
What is the stat for CS? 2.
What is the stat for BA? 1
What is the stat for OBP? 4.1
What is the stat for SLG? 2.3
What is the stat for OPS? 2.1


In [24]:
print(stats)

['3.1', '3.2', '4.1', '1.2', '3.2', '4.3', '2.1', '5', '2.5', '3.', '1.2', '3.1', '4.1', '7.1', '1.2', '1.2', '3.1', '2.', '1', '4.1', '2.3', '2.1']


In [35]:
features = np.array([stats])

prediction = classifier.predict(features)

print(prediction)


Prediction: {prediction}


/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: FutureWarning: Arrays of bytes/strings is being converted to decimal numbers if dtype='numeric'. This behavior is deprecated in 0.24 and will be removed in 1.1 (renaming of 0.26). Please convert your data to numeric values explicitly instead.
  return f(*args, **kwargs)
